# IBM Applied Data Science Capstone Course by Coursera

 
_Opening a New Shopping Mall in Pune, India

Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
Get the geographical coordinates of the neighborhoods
Obtain the venue data for the neighborhoods from Foursquare API
Explore and cluster the neighborhoods
Select the best cluster to open a new shopping mall

Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


Scrap data from Wikipedia page into a DataFrame

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Villages_in_Pune_district").text

In [3]:

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:

# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Adivare
1,Aghane
2,Ahupe
3,"Ale, Maharashtra"
4,Alegaon Paga


In [7]:
# print the number of rows of the dataframe
kl_df.shape

(137, 1)

 Get the geographical coordinates

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [10]:
coords

[[19.151500000000055, 73.68717000000004],
 [18.940170000000023, 72.83483000000007],
 [18.940170000000023, 72.83483000000007],
 [18.940170000000023, 72.83483000000007],
 [18.61936000000003, 74.33635000000004],
 [19.151500000000055, 73.68717000000004],
 [18.940170000000023, 72.83483000000007],
 [19.062230000000056, 72.88059000000004],
 [19.129060000000038, 72.84644000000003],
 [18.940170000000023, 72.83483000000007],
 [19.00496000000004, 73.94583000000006],
 [19.151500000000055, 73.68717000000004],
 [18.04916000000003, 75.03412000000003],
 [18.994750000000067, 73.96787000000006],
 [19.047420000000045, 74.12820000000005],
 [19.034980000000076, 72.86164000000008],
 [19.186960000000056, 72.86262000000005],
 [19.031690000000026, 72.84360000000004],
 [18.247850000000028, 74.18536000000006],
 [21.083330000000046, 79.33333000000005],
 [19.134280000000047, 73.72879000000006],
 [19.102280000000064, 72.84749000000005],
 [19.102280000000064, 72.84749000000005],
 [19.102280000000064, 72.847490000000

In [18]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [19]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [20]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(137, 3)


,Neighborhood,Latitude,Longitude
0,Adivare,19.151500,73.687170
1,Aghane,18.940170,72.834830
2,Ahupe,18.940170,72.834830
3,"Ale, Maharashtra",18.940170,72.834830
4,Alegaon Paga,18.619360,74.336350
5,"Amade, Ambegaon",19.151500,73.687170
6,Ambadwet,18.940170,72.834830
7,Ambedara,19.062230,72.880590
8,"Amboli, Pune",19.129060,72.846440
9,Amondi,18.940170,72.834830


In [21]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

Create a map of Mumbai with neighborhoods superimposed on top

In [22]:
# get the coordinates of Mumbai
address = 'Pune, India'

geolocator = Nominatim(user_agent="myapplication")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune, India 18.521428, 73.8544541.


In [23]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [24]:
# save the map as HTML file
map_kl.save('map_kl.html')

Use the Foursquare API to explore the neighborhoods

In [25]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your_client_id' # your Foursquare ID
CLIENT_SECRET = 'your_client_secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version




In [26]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [27]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4495, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Aghane,18.94017,72.83483,Royal China,18.938715,72.832933,Chinese Restaurant
1,Aghane,18.94017,72.83483,Town House Cafe,18.938550,72.833464,Bar
2,Aghane,18.94017,72.83483,Sher-E-Punjab,18.937944,72.837853,Indian Restaurant
3,Aghane,18.94017,72.83483,Starbucks,18.932190,72.833959,Coffee Shop
4,Aghane,18.94017,72.83483,Yazdani Bakery,18.933191,72.833591,Bakery


In [28]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aghane,100,100,100,100,100,100
Ahupe,100,100,100,100,100,100
"Ale, Maharashtra",100,100,100,100,100,100
Ambadwet,100,100,100,100,100,100
Ambedara,90,90,90,90,90,90
"Amboli, Pune",70,70,70,70,70,70
Amondi,100,100,100,100,100,100
"Apati, Ambegaon",5,5,5,5,5,5
Awasari Kh,1,1,1,1,1,1


In [29]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 177 uniques categories.


In [30]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Chinese Restaurant', 'Bar', 'Indian Restaurant', 'Coffee Shop',
       'Bakery', 'Cricket Ground', 'Parsi Restaurant', 'Café',
       'Middle Eastern Restaurant', 'Scenic Lookout',
       'Seafood Restaurant', 'Clothing Store', 'Ice Cream Shop', 'Market',
       'Park', 'Cheese Shop', 'Beach', 'Art Gallery',
       'Gym / Fitness Center', 'Hotel', 'Boutique', 'Breakfast Spot',
       'Fast Food Restaurant', 'Train Station', 'Lounge', 'Dessert Shop',
       'Flea Market', 'Pizza Place', 'College Academic Building',
       'Gift Shop', 'History Museum', 'Asian Restaurant', "Men's Store",
       'Music Store', 'Italian Restaurant', 'Diner',
       'Japanese Restaurant', 'Pub', 'General Entertainment',
       'Fried Chicken Joint', 'Restaurant', 'New American Restaurant',
       'Gastropub', 'Multiplex', 'Bookstore', 'BBQ Joint', 'Stadium',
       'Mediterranean Restaurant', 'Hockey Arena', 'Donut Shop'],
      dtype=object)

In [31]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

False

 Analyze Each Neighborhood

In [32]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(4495, 178)


,Neighborhoods,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Cave,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Tea Room,Thai Restaurant,Theater,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store
0,Aghane,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aghane,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Aghane,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Aghane,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Aghane,0,0,0,0,0,0,0,0,0,0,0,

In [33]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(78, 178)


,Neighborhoods,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Cave,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Tea Room,Thai Restaurant,Theater,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store
0,Aghane,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.01,0.020000,0.000000,0.010000,0.00,0.040000,0.00,0.060000,0.01,0.000000,0.00,0.00,0.000000,0.010000,0.01,0.020000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.0,0.00,0.01,0.030000,0.010000,0.00,0.00,0.040000,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.040000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.01,0.010000,0.00,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.01,0.01,0.000000,0.020000,0.000000,0.040000,0.000000,0.090000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.010000,0.01,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.0,0.01,0.000000,0.01,0.00,0.000000,0.00,0.00,0.01,0.000000,0.010000,0.01,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.0,0.020000,0.000000,0.00,0.00,0.000000,0.020000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000
1,Ahupe,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.01,0.020000,0.000000,0.010000,0.00,0.040000,0.0

In [34]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

12

In [35]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [36]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Aghane,0.0
1,Ahupe,0.0
2,"Ale, Maharashtra",0.0
3,Ambadwet,0.0
4,Ambedara,0.0


Cluster Neighborhoods

In [37]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [38]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [39]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Aghane,0.0,1
1,Ahupe,0.0,1
2,"Ale, Maharashtra",0.0,1
3,Ambadwet,0.0,1
4,Ambedara,0.0,1


In [40]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(78, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aghane,0.0,1,18.94017,72.83483
1,Ahupe,0.0,1,18.94017,72.83483
2,"Ale, Maharashtra",0.0,1,18.94017,72.83483
3,Ambadwet,0.0,1,18.94017,72.83483
4,Ambedara,0.0,1,19.06223,72.88059


In [41]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(78, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
38,Golegaon,0.020000,0,19.164550,72.849460
34,Ghodegaon,0.020000,0,19.164550,72.849460
33,Gavdewadi,0.024096,0,19.173700,72.952420
50,Katraj,0.015625,0,19.039760,72.867610
30,Gadewadi,0.024096,0,19.173700,72.952420
52,Kolharwadi,0.015152,0,19.190470,72.844620
11,Bhawadi,0.010000,0,19.031690,72.843600
9,Bharadi,0.010989,0,19.034980,72.861640
20,Chincholi Morachi,0.020000,0,18.985870,72.831080
5,"Amboli, Pune",0.014286,0,19.129060,72.846440


Finally, let's visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

Examine Clusters

Cluster 0

In [44]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
38,Golegaon,0.020000,0,19.16455,72.84946
34,Ghodegaon,0.020000,0,19.16455,72.84946
33,Gavdewadi,0.024096,0,19.17370,72.95242
50,Katraj,0.015625,0,19.03976,72.86761
30,Gadewadi,0.024096,0,19.17370,72.95242
52,Kolharwadi,0.015152,0,19.19047,72.84462
11,Bhawadi,0.010000,0,19.03169,72.84360
9,Bharadi,0.010989,0,19.03498,72.86164
20,Chincholi Morachi,0.020000,0,18.98587,72.83108
5,"Amboli, Pune",0.014286,0,19.12906,72.84644


Cluster 1

In [45]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Sanaswadi,0.0,1,18.952080,72.827210
44,Kadus,0.0,1,18.940170,72.834830
45,"Kalamb, Ambegaon",0.0,1,19.083510,73.967030
46,Kalwadi,0.0,1,18.994480,72.845410
47,Kanase,0.0,1,18.940170,72.834830
48,Kanse,0.0,1,19.045912,72.820943
75,Uruli Kanchan,0.0,1,18.483330,74.133330
74,Thergaon,0.0,1,18.766670,74.950000
49,Karhati,0.0,1,18.328240,74.375610
73,Talegaon Dhamdhere,0.0,1,18.676170,74.187030


Cluster 2

In [46]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
10,Bhare,0.038961,2,19.186960,72.86262
19,Chinchodi,0.040000,2,19.176765,72.85411


Observation:

Most of the shopping malls are concentrated in the central area of Pune city, with the highest number in cluster 1 and moderate number in cluster 0. On the other hand, cluster 2  has only two shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. 